In [ ]:
import scraping_class
logfile = 'log.txt'## name your log file.
connector = scraping_class.Connector(logfile)
import requests
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd
import numpy as np
import html5lib

In [ ]:
def create_df(url):
    # Requesting URL with get
    response = requests.get(url) 
    
    # Getting course_id and year direct from URL
    course_id = url.split('/')[-1]
    year = url.split('/')[-3]

    # Converting to soup
    raw_html = response.text
    soup = BeautifulSoup(raw_html,'lxml')

    # Info used to scrabe certain tables (exam and on in the right corner)
    info = soup.find_all("dl", {'class':'dl-horizontal'})
    
    # Generating error list if course page is out of match
    error = []

    # Only selecting courses with 2 tabels i.e. exactly 1 exam and the info bar to the right
    # Then appending information of each side by 'dt' and 'dd' and pairing them
    if len(info) == 2:
        comp_info = pd.DataFrame()
        cleaned_id_text = ['Course Name']
        for i in info[0].find_all('dt'):
            cleaned_id_text.append(i.text)
        cleaned_id__attrb_text = [soup.find('h1',{'class':"courseTitle"}).text]
        for i in info[0].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)
        
        # If there is a blank space the lists can't be paired. Therfore if lengths dosen't match, list will be created by siblings if sibling isen't on the initial side.
        if len(info[1].find_all('dt')) == len(info[1].find_all('dd')):
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dd'):
                cleaned_id__attrb_text.append(i.text)
        else:
            for i in info[1].find_all('dt'):
                cleaned_id_text.append(i.text)
            for i in info[1].find_all('dt'):
                if i.next_sibling in info[1].find_all('dt'):
                    cleaned_id__attrb_text.append('')
                else:
                    cleaned_id__attrb_text.append(i.next_sibling.text)

        # Appending year from URL
        cleaned_id_text.append('Year')
        cleaned_id__attrb_text.append(year)
        
        # Appending course id form URL
        cleaned_id_text.append('Course id')
        cleaned_id__attrb_text.append(course_id)
        
        # Scrabing data from workload table
        work = soup.find('ul',{'class':'list-unstyled workload clearfix'})
        workfi = work.find_all('li')
        
        # Geting 1 list of element, therfore has to be paire 1-3 2-4 etc.
        for i in range(len(workfi)):
            if i % 2 ==0:
                cleaned_id_text.append(workfi[i].text)
            else: 
                cleaned_id__attrb_text.append(workfi[i].text)
        
        # Appending content
        cleaned_id_text.append(soup.find_all('div',{'class':"course-content"})[0].find('a').text)
        cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-content"})[0].find('div').text)
        
        # Appending learning outcome
        cleaned_id_text.append(soup.find_all('div',{'class':"course-description"})[0].find('a').text)
        cleaned_id__attrb_text.append(soup.find_all('div',{'class':"course-description"})[0].find('div').text)

        comp_info['Id'] = cleaned_id_text
        comp_info['Attribute'] = cleaned_id__attrb_text
    else:
        error.append(url)
        comp_info = []
        
    return(comp_info, error)